In [1]:
from pyspark.sql import functions as sqf
from pyspark.sql import Window
from pyspark.sql.functions import col

def aggregated_data(df, top = 100):
    df_exploded = df.select('ap_id', 'site_id', 'when', sqf.explode('switchports').alias('tmp'))\
                    .select('ap_id', 'site_id', 'when', 'tmp.name', 'tmp.rx_peakbps', 'tmp.tx_peakbps')
    window_clients = Window.partitionBy("site_id", "ap_id").orderBy(sqf.col("rx_peakbps").desc())
    top_sorted = df_exploded.where(col('rx_peakbps') > 0)\
                            .select('ap_id', 'site_id', 'when', 'name', 'rx_peakbps',
                                              sqf.dense_rank().over(window_clients).alias('index'))\
                            .where(sqf.col('index') < top)
    return top_sorted


top_sorted = aggregated_data(ap_stats_df)

NameError: name 'ap_stats_df' is not defined

In [2]:

paths = ['s3a://mist-secor-production/ap-stats-analytics/ap-stats-analytics-production/dt=2019-10-21/*']
ap_stats_df = spark.read.option("basePath","s3a://mist-secor-production").parquet(*paths)
